In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [14]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [15]:
outputs = []
def layer1_hook(module, input_, output):
    outputs.append(output)
model.pool.register_forward_hook(layer1_hook)

In [16]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)


In [17]:
outputs

[tensor([[[[0.2750, 0.9467, 0.7928,  ..., 0.0000, 0.5029, 0.2756],
           [0.6124, 0.2800, 0.2498,  ..., 0.0968, 0.4424, 0.0000],
           [0.6774, 0.1471, 0.0000,  ..., 0.2947, 0.0000, 0.3717],
           ...,
           [0.8490, 0.2743, 0.0000,  ..., 0.5518, 0.0214, 0.0000],
           [0.7069, 1.2405, 0.0000,  ..., 0.2565, 0.3460, 0.2900],
           [0.3794, 0.0027, 0.7377,  ..., 1.0059, 0.3139, 1.0771]],
 
          [[0.0000, 0.6592, 0.0000,  ..., 0.4917, 0.8030, 0.2695],
           [0.2880, 0.5000, 1.2823,  ..., 0.1298, 0.6471, 1.1510],
           [0.5431, 0.9385, 0.5383,  ..., 0.7354, 0.6723, 0.3105],
           ...,
           [0.4827, 0.3933, 0.6397,  ..., 1.0751, 0.6193, 0.1940],
           [0.1701, 0.2146, 0.0000,  ..., 0.0000, 0.4992, 0.4398],
           [0.2418, 0.9148, 0.0000,  ..., 0.4110, 0.5522, 0.7373]],
 
          [[1.0056, 0.7857, 0.1515,  ..., 0.6300, 0.0897, 0.1903],
           [0.4249, 0.0000, 0.6132,  ..., 0.2972, 0.5618, 0.6869],
           [1.0067, 1.59

In [18]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [22]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [23]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


